In [37]:
from pydub import AudioSegment
from os import listdir, mkdir, path
from sklearn.model_selection import train_test_split
import pandas as pd
from tqdm import tqdm

In [38]:
audio_dir = 'audio_files'
df = pd.read_csv("birds_clean_pt.csv")

In [39]:
files = listdir(audio_dir)
df['file-name'] = df['file-name'].str.lower()
df['filename'] = df['id'].astype(str) + df['file-name'].str[-4:]
downloaded = df[df['filename'].isin(files)][['filename', 'en']]
downloaded.to_csv('downloaded.csv', index=False)
print('Not downloaded:', len(df) - len(downloaded))

train, test = train_test_split(downloaded[['filename', 'en']], stratify=downloaded["en"], test_size=0.1)

Not downloaded: 1


In [40]:
def to_wav(filename, n_channels, sample_rate, input_dir, output_dir):
    input_type = filename[-4:].lower()
    if input_type == '.mp3':
        sound = AudioSegment.from_mp3(f"{input_dir}/{filename}")
        new_fn = filename[:-4] + '.wav'
    elif input_type == '.wav':
        sound = AudioSegment.from_wav(f"{input_dir}/{filename}")
        new_fn = filename
    sound = sound.set_channels(n_channels)
    sound.set_frame_rate(sample_rate)
    sound.export(f"{output_dir}/{new_fn}", format="wav")

In [41]:
wav_dir = 'wav_files'
if not path.isdir(wav_dir):
    mkdir(wav_dir)
    mkdir(wav_dir + '/test/')
    mkdir(wav_dir + '/train/')

In [42]:
for fn in tqdm(test['filename']):
    to_wav(fn, n_channels=1, sample_rate=16000, input_dir=audio_dir, output_dir=wav_dir+'/test')

for fn in tqdm(train['filename']):
    to_wav(fn, n_channels=1, sample_rate=16000, input_dir=audio_dir, output_dir=wav_dir+'/train')

100%|██████████| 8825/8825 [1:20:31<00:00,  1.83it/s]    
